<a href="https://colab.research.google.com/github/thabir303/CSE-837-ML/blob/main/hw5/logistic-regression-lab-exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚯 Lecture 11 Lab: Logistic regression and spam detection

<img src="https://github.com/joshuagrossman/mse125-labs-public/blob/main/hw5/img/spam-email.png?raw=1" alt= “spam-email” width="500" />

## ✅ Setup and data import
In this lab, we will work with a [classic dataset](https://archive.ics.uci.edu/dataset/94/spambase) of 4,601 emails classified as spam or not spam.

In [1]:
# Load in additional functions
library(tidyverse)
library(lubridate)

# Use three digits past the decimal point,
# and don't use scientific notation.
options(digits = 3, scipen = 999)

# Format plots with a white background and dark features.
theme_set(theme_bw())

# Increase the default text size of plots.
# If you are *not* working in Google Colab, we recommend commenting
# out this line of code.
theme_update(text = element_text(size = 20))

# Increase the default plot width and height.
# If you are *not* working in Google Colab, we recommend commenting
# out this line of code.
options(repr.plot.width=12, repr.plot.height=8)

# Read in the data
spam = read_csv('https://jdgrossman.com/assets/spam.csv')

# peek at 10 random rows
sample_n(spam, 10)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Rows: 4601 Columns: 58
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (58): make, address, all, 3d, our, over, remove, internet, order, mail, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


make,address,all,3d,our,over,remove,internet,order,mail,⋯,char_semicolon,char_left_paren,char_left_bracket,char_exclamation,char_dollar,char_pound,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.00,0.00,3.03,0,0.00,0.0,0,0,0.0,0.00,⋯,0.00,0.352,0.000,2.112,0.000,0.000,3.91,11,43,1
0.00,0.00,0.00,0,0.00,0.0,0,0,0.0,0.00,⋯,0.00,0.000,0.000,0.000,0.000,0.000,1.00,1,2,1
0.00,1.25,0.00,0,0.00,0.0,0,0,0.0,0.00,⋯,0.00,0.164,0.000,0.109,0.054,0.000,2.19,18,136,0
0.00,0.00,0.00,0,0.00,0.0,0,0,0.0,0.00,⋯,0.21,0.210,0.000,0.000,0.105,0.000,1.87,22,112,1
0.26,0.05,1.45,0,0.37,0.1,0,0,0.1,0.10,⋯,0.00,0.076,0.000,0.262,0.186,0.025,11.79,289,2288,1
0.00,0.00,0.00,0,0.00,0.0,0,0,0.0,0.73,⋯,0.00,0.555,0.666,0.000,0.000,0.111,2.35,12,127,0
0.00,0.00,0.00,0,0.00,0.0,0,0,0.0,0.00,⋯,0.00,0.000,0.000,0.000,0.000,0.000,1.27,4,19,1
0.67,0.00,0.67,0,0.00,0.0,0,0,0.0,0.67,⋯,0.00,0.000,0.000,2.413,0.000,0.000,3.38,37,132,1
0.00,0.00,0.00,0,0.00,0.0,0,0,0.0,0.00,⋯,0.00,0.757,0.000,0.000,0.000,0.000,2.50,11,50,0


## ♨️ Warm up

How many emails are in the database?

What fraction of the emails in the database are spam?

Which email contains the highest percentage of words matching "money"? What percentage of words in that email match "money"?

In [ ]:
# Your code here!



## 🎲 Linear probability models (LPMs)

Fit a linear regression model to the spam data with the `lm` function.

Use the following covariates to predict the likelihood that an email is spam:
- `char_dollar`
- `credit`
- `money`
- `re`

How would you interpret the model coefficients for the intercept and for `char_dollar`?

- Note: `char_dollar` represents the percentage of characters in the email that match `$`.

In [ ]:
# Your code here!



Using your linear probability model and the `predict` function, predict the in-sample probability that each email is spam.

What is the smallest predicted probability? The largest? Do you notice any issues with these predictions?

In [ ]:
# Your code here!



## 🎰 Odds functions

Write two functions:
- A function to convert probabilities to odds.
- A function to convert odds to probabilities

Test your functions by making sure that 2:1 odds returns a 2/3 probability, and vice versa.

Finally, suppose my probability of winning is 60%. If I double my odds of winning, what is my new probability of winning?

In [ ]:
# Your code here!



## 🪙 Fitting a logistic regression model

We can fit a logistic regression model with the same covariates as above with the following code:

In [ ]:
model = glm(is_spam ~ 1 + char_dollar + credit + money + re, family='binomial', data=spam)

summary(model)

Interpret the intercept and `money` coefficients for the logistic regression model three different ways:
1. On the log odds scale
2. On the odds scale (by exponentiating the coefficients)
3. On the probability scale (using either the odds functions you wrote, or the divide by 4 trick).

Tip: Use the `coef` function to extract coefficients from the model.

In [ ]:
# Your code here!

